## Stage 1: Preprocessing & Cleaning

### Setup & Data Loading

In [1]:
# Import necessary libraries
import pandas as pd  # library for data manipulation and analysis
import re  # library for regular expressions
import os  # library for operating system dependent functionality
import string  # library for string operations
import nltk  # library for natural language processing
import spacy  # library for advanced natural language processing
import requests  # library for making HTTP requests
import contractions  # library for expanding contractions
from langdetect import detect  # library for language detection
from nltk.corpus import stopwords  # library for stop words
from nltk.stem import WordNetLemmatizer  # library for lemmatizing words
from nltk.tokenize import word_tokenize, sent_tokenize  # libraries for tokenizing text
from unidecode import unidecode  # library for converting accented characters
from bs4 import BeautifulSoup  # library for parsing HTML and XML documents
from nltk.sentiment import SentimentIntensityAnalyzer  # library for sentiment analysis
nltk.download('vader_lexicon')  # download the VADER sentiment analysis lexicon
nltk.download('wordnet')  # download WordNet for lemmatization
nltk.download('stopwords')  # download stopwords for text preprocessing
nltk.download('punkt')  # download the Punkt tokenizer for sentence segmentation
nltk.download('maxent_ne_chunker')  # download the maximum entropy chunker for named entity recognition
nltk.download('averaged_perceptron_tagger')  # download the averaged perceptron tagger for part-of-speech tagging
nltk.download('words')  # download the NLTK corpus of words


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tim/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/tim/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /Users/tim/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
# Download a spacy model, can also be adjusted (medium = en_core_web_sm, large = en_core_web_lg)
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 7.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [51]:
# Define the file path
file_path = '../data/esg_documents_for_dax_companies.csv'

# Check if the directory and file exist
if os.path.exists(file_path):
    # Read the dat
    raw_data = pd.read_csv(file_path, delimiter='|', index_col=0)
else:
    print(f'The file {file_path} does not exist, please download the file and store it in "/data".')


In [4]:
# Check loaded data and reset index
raw_data = raw_data.reset_index(drop=True)
raw_data.head(10)

,company,content,datatype,date,domain,esg_topics,internal,symbol,title,url
0,Beiersdorf AG,Sustainability Highlight Report CARE BEYOND SK...,sustainability_report,2021-03-31,NaN,"['CleanWater', 'GHGEmission', 'ProductLiabilit...",1,BEI,BeiersdorfAG Sustainability Report 2021,NaN
1,Deutsche Telekom AG,Corporate Responsibility Report 2021 2 Content...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'Iso50001', 'GlobalWarming', ...",1,DTE,DeutscheTelekomAG Sustainability Report 2021,NaN
2,Vonovia SE,VONOVIA SE SUSTAINABILITY REPORT 2021 =For a S...,sustainability_report,2021-03-31,NaN,"['Whistleblowing', 'DataSecurity', 'Vaccine', ...",1,VNA,VonoviaSE Sustainability Report 2021,NaN
3,Merck KGaA,Sustainability Report 2021 TABLE OF CONTENTS S...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'DataMisuse', 'DrugResistance...",1,MRK,MerckKGaA Sustainability Report 2021,NaN
4,MTU,Our ideas and concepts FOR A SUSTAINABLE FUTUR...,sustainability_report,2020-03-31,NaN,"['WorkLifeBalance', 'Corruption', 'AirQuality'...",1,MTX,MTUAeroEngines Sustainability Report 2020,NaN
5,E ONSE,#StandWithUkraine Sustainability Report 2021 C...,sustainability_report,2021-03-31,NaN,"['DataSecurity', 'Iso50001', 'GlobalWarming', ...",1,EOAN,E.ONSE Sustainability Report 2021,NaN
6,RWE AG,Focus on tomorrow. Sustainability Report 2021 ...,sustainability_report,2021-03-31,NaN,"['WorkLifeBalance', 'Corruption', 'Iso50001', ...",1,RWE,RWEAG Sustainability Report 2021,NaN
7,Heidelberg Cement AG,Annual Report 2021 HeidelbergCement at a glanc...,annual_report,2021-03-31,NaN,"['WorkLifeBalance', 'Vaccine', 'DataSecurity',...",1,HEI,HeidelbergCementAG Annual Report 2021,NaN
8,Heidelberg Cement AG,Company Strategy & Business & Product & Produc...,sustainability_report,2020-03-31,NaN,"['CleanWater', 'Corruption', 'Whistleblowing',...",1,HEI,HeidelbergCementAG Sustainability Report 2020,NaN
9,Siemens AG,Sustainability 1 Siemens 2 Our 3 Governance – ...,sustainability_report,2020-03-31,NaN,"['DataSecurity', 'Iso50001', 'EmployeeTurnover...",1,SIE,SiemensAG Sustainability Report 2020,NaN


**Column descriptions**
- symbol: stock symbol of the company
- company: company name
- date: publication date of document
- title: document title
- content: document content
- datatype: document type
- internal: is this a report by company (1) or a third-party document (0)
- domain (optional): Web domain where the document was published
- url (optional): URL where the document can be accessed
- esg_topics (optional): ESG topics extracted from the data using our internal NLP

In [5]:
# Check shape (row and column amount)
raw_data.shape

(11188, 10)

In [6]:
# Check datatypes
raw_data.dtypes

company       object
content       object
datatype      object
date          object
domain        object
esg_topics    object
internal       int64
symbol        object
title         object
url           object
dtype: object

The data is loaded correctly.

In [7]:
def csv_checkpoint(df, filename='checkpoint'):
    """
    Saves a DataFrame to a CSV file and loads it back into a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame to save and load.
        filename (str): The name of the CSV file to save the DataFrame to (default: 'checkpoint').

    Returns:
        pandas.DataFrame: The loaded DataFrame.
    """
    if not os.path.exists('../data/checkpoints/'):  # Check if the directory exists and create it if it doesn't
        os.makedirs('../data/checkpoints/')

    # Save DataFrame to CSV
    df.to_csv(f'../data/checkpoints/{filename}.csv', index=False, sep='|')  # Save DataFrame to CSV with specified filename
    print(f'Saved DataFrame to {filename}.csv')

    # Load CSV back into DataFrame
    df = pd.read_csv(f'../data/checkpoints/{filename}.csv', delimiter='|')  # Load CSV back into DataFrame
    print(f'Loaded DataFrame from {filename}.csv')

    return df


## General Data Cleaning

As initial data cleaning steps, the following is conducted:
- Rows with missing "content" were dropped to prevent any missing data-related issues. Missing data can create gaps in the data and lead to errors or distortions in the analysis.
- The "URL" column was removed as the relevant information was available in the "domain" column. Removing redundant columns simplifies the data set and makes it easier to work with
- Duplicate entries were identified and removed, resulting in a cleaner and more concise dataset. Duplicates can distort the data and lead to biased analysis. 
- Language checking was conducted and all rows with non-English content were dropped to ensure consistent language. Language inconsistencies can create bias in the data and lead to inaccurate conclusions. Therefore, it is important to ensure that the data is consistent in language to prevent linguistic biases.
- "Date" is formatted as a date and wrong dates, e.g. "bayer-03-31" are replaced with a default date (2023-03-31).
- Remove company name parts like "AG" for clarity
- The "sample" method was used to check the data for representativeness and potential issues.

In [8]:
general_cleaned_data = raw_data.copy(deep=True)

In [9]:
# Delete all rows with no content, e.g. no report
general_cleaned_data = general_cleaned_data.dropna(subset=['content'])

In [10]:
# Drop the "url" column, since the most relevant information from an analysis perspective is already in the "domain" column (e.g. the source of the report)
general_cleaned_data = general_cleaned_data.drop(columns=['url'])

In [11]:
# Check for duplicates and delete them
duplicates = general_cleaned_data[general_cleaned_data.duplicated()]
print(f'Duplicated rows: {len(duplicates)}')
general_cleaned_data = general_cleaned_data.drop_duplicates()

Duplicated rows: 6


In [12]:
# Check for other languange than English
general_cleaned_data['language'] = general_cleaned_data['content'].apply(lambda x: detect(x))
not_english = len(general_cleaned_data) - len(general_cleaned_data.loc[general_cleaned_data['language'] == 'en'])

# Drop rows with other languange, since other languanges influences to quality of the later analysis
general_cleaned_data = general_cleaned_data.loc[general_cleaned_data['language'] == 'en']

print(f'Deleted amount of rows with language other ehan English: {not_english}')
general_cleaned_data.drop(['language'], axis=1, inplace=True)

Deleted amount of rows with language other ehan English: 106


In [13]:
# Correct the dates to ISO standard
def find_incorrect_dates(data):
    incorrect_dates = []

    for index, row in data.iterrows():
        try:
            pd.to_datetime(row['date'], format='%Y-%m-%d', errors='raise')
        except ValueError:
            incorrect_dates.append((index, row['date']))

    return incorrect_dates

incorrect_date_rows = find_incorrect_dates(general_cleaned_data)
print("Incorrectly formatted dates:")
for index, date in incorrect_date_rows:
    print(f"Row {index}: {date}")

Incorrectly formatted dates:
Row 13: p.DE-03-31
Row 18: p.DE-03-31
Row 20: bayer-03-31
Row 22: p.DE-03-31
Row 25: p.DE-03-31
Row 26: p.DE-03-31
Row 31: p.DE-03-31
Row 32: p.DE-03-31
Row 33: p.DE-03-31
Row 37: p.DE-03-31
Row 41: p.DE-03-31
Row 50: p.DE-03-31
Row 78: p.DE-03-31
Row 80: p.DE-03-31
Row 86: p.DE-03-31
Row 87: p.DE-03-31
Row 88: p.DE-03-31


In [14]:
# Correct the wrong formatted dates and set default date
def correct_date_format(data):
    data['date'] = pd.to_datetime(data['date'], errors='coerce').fillna('2022-03-31')
    return data

general_cleaned_data = correct_date_format(general_cleaned_data)

In [15]:
# Replace company name parts like "AG" to have a cleaner name
general_cleaned_data['company'] = general_cleaned_data['company'].str.replace(' AG', '')
general_cleaned_data['company'] = general_cleaned_data['company'].str.replace(' SE', '')
general_cleaned_data['company'] = general_cleaned_data['company'].str.replace(' KGaA', '')

In [16]:
# Drop rows with no content, e.g. no report
general_cleaned_data = general_cleaned_data.dropna(subset=['content'])

In [45]:
# Check the date with some samples
general_cleaned_data.sample(5)

,company,content,datatype,date,domain,esg_topics,internal,symbol,title
1983,BMW,Laser specialist TRUMPF has for the first time...,business,2022-07-04,marketscreener,['CarbonDioxide'],0,BMW,Vitesco Technologies: Increased sustainability...
1022,Airbus,European aircraft manufacturer Airbus has spar...,tech,2022-02-08,airport-technology,"['Privacy', 'extremism']",0,AIR,Airbus warns it could leave the UK in case of ...
5265,E ONSE,Cost and carbon savings from electric vehicles...,general,2022-11-10,edie,"['Environment', 'FossilFuels', 'EMobility', 'G...",0,EOAN,"Convenience is key, says E.ON in new drive to ..."
10430,Volkswagen,Volkswagen's IT board member Hauke Stars belie...,business,2022-12-22,marketscreener,['Environment'],0,VOW3,VW IT boss after Conti hack: business-critical...
6651,Merck,You are currently accessing Investment Week vi...,business,2022-10-14,investmentweek,"['CustomerService', 'Governance']",0,MRK,Deep Dive: Will newer or older artists win the...


In [18]:
# Change name of "Muenchener Rueckversicherungs Gesellschaft AGin Muenchen" to something more readable
general_cleaned_data['company'] = general_cleaned_data['company'].replace('Muenchener Rueckversicherungs Gesellschaftin Muenchen', 'Munich R')

In [19]:
# Create checkpoint file
general_cleaned_data = csv_checkpoint(general_cleaned_data, 'general_cleaned_data')

Saved DataFrame to general_cleaned_data.csv
Loaded DataFrame from general_cleaned_data.csv


## Text Data Cleaning & Preprocessing

The "content" column, containing the text of the reports, undergoes a series of cleaning, normalization, and preprocessing steps to ensure accurate and efficient analysis. These steps include:

- **String conversion**: Converting the input to a string format ensures consistency and compatibility during subsequent processing tasks.
- **Lowercase conversion**: Transforming all text to lowercase serves as a simple normalization step, reducing the complexity and variability of the input data.
- **Unicode decoding**: Removing diacritics (e.g., accented characters) and normalizing the text encoding mitigates potential discrepancies arising from different encoding formats.
- **URL and email address removal**: Eliminating URLs and email addresses reduces noise in the dataset, as these elements do not contribute valuable information for the analysis.
- **Extra whitespace removal**: Eradicating extra whitespaces improves text analysis and tokenization by ensuring that only meaningful spaces are retained.
- **Contact detail removal**: Excluding phone numbers, contact person strings, and social media references further minimizes noise in the dataset, honing the focus on relevant text.
- **Table of contents removal**: Discarding the table of contents enhances the data quality by eliminating repetitive and non-essential information.
- **Named entity removal**: Employing the spaCy model to remove human names and other named entities optimizes the text for analysis and modeling by concentrating on pertinent content.
- **Abbreviation expansion**: Utilizing the contractions library and custom functions with regular expressions, common and uncommon abbreviations are expanded to improve text interpretation.
- **Special character elimination**: Excluding all special characters, except punctuation, refines the input data. Retaining punctuation is necessary for accurate sentence tokenization and removed after sentence tokenization..
- **Tokenization and lemmatization**: Tokenizing words and sentences, and subsequently lemmatizing words using the WordNetLemmatizer from nltk, streamlines the text and reduces morphological variations.
- **Stopword removal**: Customizing the nltk stopwords list by adding or removing specific stopwords enables more precise and tailored text analysis.
- **Part-of-speech (POS) tagging**: Assigning POS tags to words and sentences enhances the text representation by providing additional linguistic information, which may be beneficial for subsequent analysis and modeling tasks.
- **Sentiment Analysis**: Basic sentiment value calculation on the tokenzued sentences to get first insights im terms of the sentiments in the reports within the EDA.

Spellchecking was tested with TextBlob and PySpellChecker but deliverd not useful results

In [20]:
cleaned_data = general_cleaned_data.copy(deep=True)

In [21]:
# Since the spacy model shows better results on the "raw" text, the named entity removal is conducted before all normalization and cleaning steps
spacy_model = spacy.load('en_core_web_md')
spacy_model.max_length = 1800000 # Increase max text length

def remove_named_entities(text):
    """
    Removes named entities from text and returns the modified text and the count of named entities removed.

    Args:
        text (str): The text to remove named entities from.

    Returns:
        tuple: A tuple containing the modified text (str) and the count of named entities removed (int).
    """
    doc = spacy_model(text)
    
    named_entities = set()
    for ent in doc.ents:
        if ent.label_ in ["PERSON"]:
            named_entities.add(ent.text)
    
    named_entities_count = len(named_entities)
    
    for named_entity in named_entities:
        text = text.replace(named_entity, '')
    
    return text, named_entities_count

# Assuming cleaned_data is a pandas DataFrame with a 'content' column
cleaned_data['cleaned_content'], name_entity_count = zip(*cleaned_data['content'].apply(remove_named_entities))
print("Name entities removed:", sum(name_entity_count))

Name entities removed: 94745


In [22]:
def remove_urls(text):
    urls = re.findall(r'http\S+|www\S+|https\S+', text, flags=re.MULTILINE)
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE), len(urls)

def remove_emails(text):
    mail_addresses = re.findall(r'\S+@\S+\s?', text, flags=re.MULTILINE)
    return re.sub(r'\S+@\S+\s?', '', text, flags=re.MULTILINE), len(mail_addresses)

def remove_extra_whitespace(text):
    extra_spaces = re.findall(r'\s{2,}', text)
    return re.sub(r'\s+', ' ', text).strip(), len(extra_spaces)

cleaned_data['cleaned_content'] = cleaned_data['cleaned_content'].astype(str) # Convert all texts to string
cleaned_data['cleaned_content'] = cleaned_data['cleaned_content'].apply(lambda x: x.lower()) # Convert all texts to lower-case
cleaned_data['cleaned_content'] = cleaned_data['cleaned_content'].apply(lambda x: unidecode(x, errors="preserve")) # Remove diacritics / accented characters and unicode normalization
cleaned_data['cleaned_content'], url_count = zip(*cleaned_data['cleaned_content'].apply(remove_urls)) # Remove URLs from texts
cleaned_data['cleaned_content'], email_count = zip(*cleaned_data['cleaned_content'].apply(remove_emails)) # Remove e-mail addresses from texts
cleaned_data['cleaned_content'], extra_space_count = zip(*cleaned_data['cleaned_content'].apply(remove_extra_whitespace)) # Remove extra whitespaces from texts

print("URLs removed:", sum(url_count))
print("Mail addresses removed:", sum(email_count))
print("Extra whitespaces removed:", sum(extra_space_count))

URLs removed: 7496
Mail addresses removed: 434
Extra whitespaces removed: 149381


In [23]:
def remove_contact_details(text):
    # Remove phone numbers
    phone_regex = r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'
    phone_count = len(re.findall(phone_regex, text))
    text = re.sub(phone_regex, '', text)

    # Remove common contact-related phrases
    contact_phrases_regex = r'\b(?:Contact Person|Phone|Tel|Fax|Mobile|E?mail|Skype|Twitter|Facebook|LinkedIn|Website):\b'
    contact_phrases_count = len(re.findall(contact_phrases_regex, text, flags=re.IGNORECASE))
    text = re.sub(contact_phrases_regex, '', text, flags=re.IGNORECASE)

    total_count = phone_count + contact_phrases_count
    return text, total_count

def remove_table_of_contents(text):
    # Remove common table of contents phrases
    toc_phrases_regex = r'\b(?:Table of Contents|Contents)\b'
    toc_phrases_count = len(re.findall(toc_phrases_regex, text, flags=re.IGNORECASE))
    text = re.sub(toc_phrases_regex, '', text, flags=re.IGNORECASE)

    # Remove content with numbering like "1. Introduction", "1.1. Background", "A. Overview", etc.
    toc_entries_regex = r'(^|\n)\s*\w+(\.\w+)*\s+\w+([\w\s]+)?'
    toc_entries_count = len(re.findall(toc_entries_regex, text))
    text = re.sub(toc_entries_regex, '', text)

    total_count = toc_phrases_count + toc_entries_count
    return text, total_count

cleaned_data['cleaned_content'], contact_count = zip(*cleaned_data['cleaned_content'].apply(remove_contact_details))
cleaned_data['cleaned_content'], toc_count = zip(*cleaned_data['cleaned_content'].apply(remove_table_of_contents))
print("Contact information removed:", sum(contact_count))
print("TOCs removed:", sum(toc_count))

Contact information removed: 44260
TOCs removed: 9607


In [24]:
def expand_contractions(text):
    expanded_text = []
    for word in text.split():
        expanded_text.append(contractions.fix(word))
    expanded_text = ' '.join(expanded_text)
    return contractions.fix(expanded_text)

cleaned_data['cleaned_content'] = cleaned_data['cleaned_content'].apply(expand_contractions)

In [25]:
# Expand custom abbreviations which are not captured by "contractions"
# Basic idea from: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
# Compile the regular expressions only once for efficiency
specific_patterns = [
    (re.compile(r"won['’]t"), "will not"),
    (re.compile(r"can['’]t"), "can not"),
]

def decontracted(phrase):
    """
    Expands contractions in a given phrase and returns the modified phrase and the count of contractions expanded.

    Args:
        phrase (str): The phrase to expand contractions in.

    Returns:
        tuple: A tuple containing the modified phrase (str) and the count of contractions expanded (int).
    """
    count = 0

    # Replace specific patterns
    for pattern, replacement in specific_patterns:
        matches = len(pattern.findall(phrase))
        count += matches
        phrase = pattern.sub(replacement, phrase)

    return phrase, count

# Apply the function to expand abbreviations
cleaned_data['cleaned_content'], abbreviation_counts = zip(*cleaned_data['cleaned_content'].apply(decontracted))
print("Expanded custom abbreviations:", sum(abbreviation_counts))

Expanded custom abbreviations: 0


In [26]:
# Remove special characters excl. punctuation since this is needed by the sentence tokenization
def remove_non_alphanumeric(text, remove_punctuation=False):
    if remove_punctuation:
        pattern = r'[^a-zA-Z0-9\s]'
    else:
        pattern = r'[^a-zA-Z0-9\s.,!?\'"]'
    
    special_chars = re.findall(pattern, text)
    return re.sub(pattern, '', text), len(special_chars)

cleaned_data['cleaned_content'], special_char_count = zip(*cleaned_data['cleaned_content'].apply(remove_non_alphanumeric, remove_punctuation=False))
print("Special characters excl. punctuation removed:", sum(special_char_count))

Special characters excl. punctuation removed: 1388738


In [27]:
def tokenize_words(text):
    # Remove numbers, digits, and punctuation
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize words
    tokens = word_tokenize(text)

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens, len(tokens)

cleaned_data['word_tokens'], word_token_count = zip(*cleaned_data['cleaned_content'].apply(tokenize_words))
print("Generated word token amount:", sum(word_token_count))

Generated word token amount: 16963356


In [28]:
def tokenize_sentences(text):
    # Tokenize sentences
    tokens = sent_tokenize(text)
    
    return tokens, len(tokens)

cleaned_data['sentence_tokens'], sentence_token_count = zip(*cleaned_data['cleaned_content'].apply(tokenize_sentences))
print("Generated sentence token amount:", sum(sentence_token_count))

Generated sentence token amount: 687669


In [29]:
def remove_stopwords_from_word_tokens(tokens, custom_stopwords):
    """
    Removes stopwords and one-character tokens from a list of word tokens and returns the modified list and the count of removed items.

    Args:
        tokens (list): The list of word tokens to remove stopwords from.
        custom_stopwords (list): A list of custom stopwords to remove from the word tokens.

    Returns:
        tuple: A tuple containing the modified list of word tokens (list) and the count of removed items (int).
    """
    filtered_tokens = [
        token for token in tokens
        if token.lower() not in custom_stopwords and len(token) > 1
    ]
    
    return filtered_tokens, len(tokens) - len(filtered_tokens)

def remove_stopwords_from_sentence_tokens(sentences_list, custom_stopwords):
    """
    Removes stopwords, one-character tokens, digits, numbers, and special characters (excluding whitespace) from a list of sentence tokens and returns the modified list and the count of removed items.

    Args:
        sentences_list (list): The list of sentence tokens to remove stopwords from.
        custom_stopwords (list): A list of custom stopwords to remove from the sentence tokens.

    Returns:
        tuple: A tuple containing the modified list of sentence tokens (list) and the count of removed items (int).
    """
    filtered_sentences_list = []
    total_removed_items_count = 0

    for sentence in sentences_list:
        # Tokenize the sentence into words
        word_tokens = word_tokenize(sentence)

        # Remove stopwords, one-character tokens, digits, numbers, and special characters (excluding whitespace) from word tokens
        filtered_word_tokens = [
            re.sub(rf"[{re.escape(string.punctuation)}]", '', token) for token in word_tokens
            if token.lower() not in custom_stopwords
            and len(token) > 1
            and not re.search(r'\d', token)
            and not re.search(r'\W', token)
        ]

        # Reconstruct the sentence without the removed words and special characters
        filtered_sentence = ' '.join(filtered_word_tokens)
        removed_items_count = len(word_tokens) - len(filtered_word_tokens)
        filtered_sentences_list.append(filtered_sentence)
        total_removed_items_count += removed_items_count

    return filtered_sentences_list, total_removed_items_count

# Define custom stopwords to add or remove (the extra stopwords were identified by the TFIDF based wordcloud)
custom_stopwords = {
    'add': ['said','company','companies','year','billion','million','siemens','linde','rwe','volkswagen','symrise','porsche','sap','adidas','puma','airbus','bmw','hannover','mtu','heiderbergcement','qiagen','benz','continental','bayer','fresenius','wa', 'ha', 'eur', 'allianz', 'board'],
    'remove': [''] # Currently not needed
}

# Combine stopwords to filter the content of the reports
all_stopwords = set(stopwords.words('english'))
all_stopwords |= set(custom_stopwords['add'])
all_stopwords -= set(custom_stopwords['remove'])

cleaned_data['word_tokens'], stopword_count_words = zip(*cleaned_data['word_tokens'].apply(remove_stopwords_from_word_tokens, custom_stopwords=all_stopwords))
cleaned_data['sentence_tokens'], stopword_count_sentences = zip(*cleaned_data['sentence_tokens'].apply(remove_stopwords_from_sentence_tokens, custom_stopwords=all_stopwords))

print("Removed stopwords in word tokens", sum(stopword_count_words))
print("Removed stopwords in sentence tokens", sum(stopword_count_sentences))

Removed stopwords in word tokens 6852067
Removed stopwords in sentence tokens 9661393


In [30]:
# Update the cleaned content based on the cleaned word tokens
cleaned_data['cleaned_content'] = cleaned_data['word_tokens'].apply(lambda x: ' '.join(x))

In [31]:
def pos_tagging_tokens(word_tokens, sentence_list):
    """
    Performs POS tagging on a given list of word tokens and a list of sentence tokens and returns the POS tagged word tokens and POS tagged sentence tokens.

    Args:
        word_tokens (list): The list of word tokens to perform POS tagging on.
        sentence_list (list): The list of sentence tokens to perform POS tagging on.

    Returns:
        tuple: A tuple containing the POS tagged word tokens (list) and the POS tagged sentence tokens (list of lists).
    """
    
    # POS tagging for word tokens
    pos_tagged_word_tokens = nltk.pos_tag(word_tokens)

    # Create a dictionary to map word tokens to their POS tags, this reduces the effort to call nltk.pos_tag twice
    pos_tags_dict = dict(pos_tagged_word_tokens)

    # POS tagging for sentence tokens
    pos_tagged_sentence_list = []
    for sentence in sentence_list:
        tokenized_sentence = nltk.word_tokenize(sentence)
        pos_tagged_sentence = [(token, pos_tags_dict[token]) for token in tokenized_sentence if token in pos_tags_dict]
        pos_tagged_sentence_list.append(pos_tagged_sentence)

    return pos_tagged_word_tokens, pos_tagged_sentence_list

# Apply POS tagging
pos_tags = cleaned_data.apply(lambda row: pos_tagging_tokens(row['word_tokens'], row['sentence_tokens']), axis=1)
cleaned_data['pos_tagged_word_tokens'], cleaned_data['pos_tagged_sentence_tokens'] = zip(*pos_tags)

In [32]:
# Create checkpoint file
cleaned_data = csv_checkpoint(cleaned_data, 'cleaned_data')

Saved DataFrame to cleaned_data.csv
Loaded DataFrame from cleaned_data.csv


## Data Enrichment

Several additional information could be helpful in the further analysis, which are not included in the dataset. Therefore a small scraper is used to enrich the the dataset with the sector, industry and market capitalization of the DAX companies.

In [33]:
url = 'https://disfold.com/stock-index/dax/companies/'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table')
scraped_data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    scraped_data.append(cols)

def clean_scraped_data(data):
    cleaned_data = []
    
    for row in data:
        # Remove empty rows
        if len(row) > 0:
            # Remove the '$' and ',' signs from the market cap and convert it to float
            market_cap = float(row[3].replace('$', '').replace(',', '').replace('B', ''))
            cleaned_data.append([row[1], row[2], market_cap, row[4], row[5], row[6]])
    
    df = pd.DataFrame(cleaned_data, columns=['company_name', 'symbol', 'market_cap_in_usd_b', 'country', 'sector', 'industry'])
    
    return df

company_enrichments = clean_scraped_data(scraped_data)
company_enrichments.to_csv('../data/dax_company_sectors.csv', index=False)
company_enrichments.head()

,company_name,symbol,market_cap_in_usd_b,country,sector,industry
0,Linde plc,LIN,156.93,United Kingdom,Basic Materials,Specialty Chemicals
1,SAP SE,SAP,121.03,Germany,Technology,Software—Application
2,Siemens AG,SIE,110.13,Germany,Industrials,Specialty Industrial Machinery
3,Deutsche Telekom AG,DTE,101.78,Germany,Communication Services,Telecom Services
4,Airbus SE,AIR,96.87,Netherlands,Industrials,Aerospace & Defense


In [34]:
# Fix the ticker symbols to prevent NaN and ensure correct join conditions
company_enrichments['symbol'] = company_enrichments['symbol'].replace('SRT3', 'SRT')
company_enrichments['symbol'] = company_enrichments['symbol'].replace('HEN3', 'HNK')
company_enrichments.loc[company_enrichments['company_name'] == 'Mercedes-Benz Group AG', 'symbol'] = 'DAI'

In [35]:
cleaned_data['symbol'] = cleaned_data['symbol'].astype(pd.StringDtype())
company_enrichments['symbol'] = company_enrichments['symbol'].astype(pd.StringDtype())

# Merge the cleaned data with the enrichment
enriched_cleaned_data = pd.merge(cleaned_data, company_enrichments, how='left', on='symbol')

In [36]:
enriched_cleaned_data[enriched_cleaned_data['industry'].isnull()]

,company,content,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,sentence_tokens,pos_tagged_word_tokens,pos_tagged_sentence_tokens,company_name,market_cap_in_usd_b,country,sector,industry
48,Hannover R,Sustainability Report 2020 We face up to futur...,sustainability_report,2020-03-31,NaN,"['Whistleblowing', 'Vaccine', 'Corruption', 'G...",1,HNR1,HannoverRückversicherungAG Sustainability Repo...,approach purpose value reflect robust corporat...,"['approach', 'purpose', 'value', 'reflect', 'r...","['', 'approach', 'purpose values reflect robus...","[('approach', 'NN'), ('purpose', 'JJ'), ('valu...","[[], [('approach', 'NN')], [('purpose', 'JJ'),...",NaN,NaN,NaN,NaN,NaN
76,Hannover R,Annual Report An overview Gross premium E 01 i...,annual_report,2021-03-31,NaN,"['Vaccine', 'Monopolization', 'Corruption', 'G...",1,HNR1,HannoverRückversicherungAG Annual Report 2021,group net income policyholder surplus book vue...,"['group', 'net', 'income', 'policyholder', 'su...","['group net income policyholders', 'surplus bo...","[('group', 'NN'), ('net', 'JJ'), ('income', 'N...","[[('group', 'NN'), ('net', 'JJ'), ('income', '...",NaN,NaN,NaN,NaN,NaN


Hannover R AG cannot be matched, since it is not present in the scraped data. Since there are only 2 records this is negligible and will be fixed manually.

In [37]:
enriched_cleaned_data.loc[enriched_cleaned_data['company'] == 'Hannover R', 'sector'] = 'Financials'
enriched_cleaned_data.loc[enriched_cleaned_data['company'] == 'Hannover R', 'industry'] = 'Insurance—Reinsurance'

In [38]:
# Drop redundant columns/data
enriched_cleaned_data = enriched_cleaned_data.drop(columns=['content', 'company_name', 'country'])

In [39]:
# Check final dataframe
enriched_cleaned_data.sample(10)

,company,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,sentence_tokens,pos_tagged_word_tokens,pos_tagged_sentence_tokens,market_cap_in_usd_b,sector,industry
9113,Siemens,general,2021-10-03,itnonline,"['Cybersecurity', 'Social', 'GenderDiversity']",0,SIE,Siemens Healthineers Demonstrates Artificial I...,healthcare information management system socie...,"['healthcare', 'information', 'management', 's...",['healthcare information management systems so...,"[('healthcare', 'NN'), ('information', 'NN'), ...","[[('healthcare', 'NN'), ('information', 'NN'),...",110.13,Industrials,Specialty Industrial Machinery
5826,Infineon Technologies,tech,2021-07-12,smart2zero,['EMobility'],0,IFX,Power MOSFET packages for high current designs,ruggedness extended lifetime ask would right p...,"['ruggedness', 'extended', 'lifetime', 'ask', ...","['ruggedness extended lifetime', 'ask would ri...","[('ruggedness', 'NN'), ('extended', 'VBD'), ('...","[[('ruggedness', 'NN'), ('extended', 'VBD'), (...",41.33,Technology,Semiconductors
9459,Siemens Energy,general,2023-02-28,energyintel,"['Electrolysis', 'Compliance', 'RenewableEnerg...",0,ENR,Industry Mixed Over EU's New Green Hydrogen Rules,recently released proposed rule set renewable ...,"['recently', 'released', 'proposed', 'rule', '...",['recently released proposed rule set renewabl...,"[('recently', 'RB'), ('released', 'VBN'), ('pr...","[[('recently', 'RB'), ('released', 'VBN'), ('p...",13.70,Utilities,Utilities—Independent Power Producers
915,Airbus,tech,2022-07-27,simpleflying,"['Transparency', 'Environment']",0,AIR,Airbus Updates 2022 Delivery Estimate To 700 A...,say aim deliver around aircraft slightly fewer...,"['say', 'aim', 'deliver', 'around', 'aircraft'...","['', 'says aims deliver around aircraft slight...","[('say', 'VB'), ('aim', 'NN'), ('deliver', 'NN...","[[], [('deliver', 'VB'), ('around', 'IN'), ('a...",96.87,Industrials,Aerospace & Defense
11007,Zalando,general,2021-05-28,textileworld,"['SustainableFibre', 'ValueChain', 'Sustainabl...",0,ZAL,"Sustainable Apparel Coalition, Higg Launch New...",amsterdam may today sustainable apparel coalit...,"['amsterdam', 'may', 'today', 'sustainable', '...",['amsterdam may today sustainable apparel coal...,"[('amsterdam', 'NN'), ('may', 'MD'), ('today',...","[[('amsterdam', 'NN'), ('may', 'MD'), ('today'...",10.41,Consumer Discretionary,Internet Retail
7565,Qiagen,general,2021-04-26,livescience,"['Diversity', 'Aquaculture', 'Compliance', 'Be...",0,QIA,Pupal cannibalism by worker honey bees contrib...,com using browser version limited support cs o...,"['com', 'using', 'browser', 'version', 'limite...","['', 'using browser version limited support cs...","[('com', 'NN'), ('using', 'VBG'), ('browser', ...","[[], [('using', 'VBG'), ('browser', 'NN'), ('v...",11.38,Healthcare,Diagnostics & Research
8786,Siemens,general,2021-10-03,itnonline,"['Cybersecurity', 'Oversight', 'Social']",0,SIE,Why We Have to Pay Attention to AI Right Now,run consulting service freiherr group snapshot...,"['run', 'consulting', 'service', 'freiherr', '...","['', 'runs consulting service freiherr group',...","[('run', 'VB'), ('consulting', 'VBG'), ('servi...","[[], [('consulting', 'VBG'), ('service', 'NN')...",110.13,Industrials,Specialty Industrial Machinery
6915,Merck,tech,2022-12-06,biopharmadive,"['HumanCapital', 'Recruiting']",0,MRK,"With $ 81M, Entact Bio takes a new approach to...",otherwise caught pathology disease contrast tr...,"['otherwise', 'caught', 'pathology', 'disease'...","['otherwise caught pathology disease', 'contra...","[('otherwise', 'RB'), ('caught', 'VBN'), ('pat...","[[('otherwise', 'RB'), ('caught', 'VBN'), ('pa...",87.64,Healthcare,Drug Manufacturers—Specialty & Generic
5620,Heidelberg Cement,business,2021-07-21,dezeen,"['ClimateTech', 'CarbonCaptureAndStorage', 'Ca...",0,HEI,Norway begins `` absolutely necessary '' proje...,EUR1 project could bury vast amount captured i...,"['EUR1', 'project', 'could', 'bury', 'vast'

## Calculate Sentiment Value with Polarity Score

As a last preprocessing step, the sentiment is calculated with the (quite basic) SentimentIntensityAnalyzer.

In [40]:
sia = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    """
    Computes the sentiment score for a given text.

    Args:
        text (str): The text to compute the sentiment score for.

    Returns:
        float: The sentiment score of the text as a float between -1 and 1.
    """
    
    if not isinstance(text, str):
        return 0

    # Remove brackets and quotes, then split the text by commas
    text = text.replace("[", "").replace("]", "").replace("'", "")
    sentences = text.split(", ")

    # Compute sentiment scores for each sentence and store them in a list
    sentiment_scores = [sia.polarity_scores(sentence)['compound'] for sentence in sentences]

    # Compute the average sentiment score
    avg_sentiment_score = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0

    return avg_sentiment_score

# Sentiment score calculation provided most "balanced" results with averaged sentence tokens. Therefore the sentiment is calculated on these texts.
enriched_cleaned_data['sentiment_value'] = enriched_cleaned_data['sentence_tokens'].apply(get_sentiment_score)

In [41]:
# Check NaN rows
enriched_cleaned_data[enriched_cleaned_data['cleaned_content'].isna()]

,company,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,sentence_tokens,pos_tagged_word_tokens,pos_tagged_sentence_tokens,market_cap_in_usd_b,sector,industry,sentiment_value
1574,AkzoNobelNV,general,2021-09-09,chemistryworld,['AnimalTesting'],0,BAS,Non-animal test for skin sensitisation gets OE...,NaN,[],[],[],[],48.10,Basic Materials,Chemicals,0.0
3881,Deutsche Bank,general,2022-11-09,law360,[],0,DBK,Former Deutsche Bank Exec Says Discrimination ...,NaN,[],[''],[],[[]],24.97,Financials,Banks,0.0
3995,Deutsche Bank,business,2021-08-20,globalcapital,[],0,DBK,Deutsche Bank debuts green label in Formosa fo...,NaN,[],[''],[],[[]],24.97,Financials,Banks,0.0
7858,RWE,general,2022-10-03,energyintel,['RenewableEnergy'],0,RWE,Germany's RWE Expands US Green Power Portfolio,NaN,[],[''],[],[[]],29.97,Utilities,Utilities—Diversified,0.0


In [42]:
# Drop NaN rows
enriched_cleaned_data = enriched_cleaned_data.dropna(subset=['cleaned_content'])

In [43]:
# Create checkpoint file for further analysis
enriched_cleaned_data = csv_checkpoint(enriched_cleaned_data, 'enriched_cleaned_data')

Saved DataFrame to enriched_cleaned_data.csv
Loaded DataFrame from enriched_cleaned_data.csv


In [58]:
# Check the data after full preprocessing
enriched_cleaned_data.sample(10)

,company,datatype,date,domain,esg_topics,internal,symbol,title,cleaned_content,word_tokens,sentence_tokens,pos_tagged_word_tokens,pos_tagged_sentence_tokens,market_cap_in_usd_b,sector,industry,sentiment_value
1027,Airbus,tech,2021-03-05,aithority,['Compliance'],0,AIR,Mirakl Listed as a Representative Vendor in Ga...,mirakl industry first advanced enterprise mark...,"['mirakl', 'industry', 'first', 'advanced', 'e...",['mirakl industry first advanced enterprise ma...,"[('mirakl', 'NN'), ('industry', 'NN'), ('first...","[[('mirakl', 'NN'), ('industry', 'NN'), ('firs...",96.87,Industrials,Aerospace & Defense,0.390390
891,Airbus,general,2022-07-28,businesswire,['CarbonDioxide'],0,AIR,Aviation Capital Group Announces Delivery of O...,photo business wire aviation capital group ann...,"['photo', 'business', 'wire', 'aviation', 'cap...",['photo business wire aviation capital group a...,"[('photo', 'NN'), ('business', 'NN'), ('wire',...","[[('photo', 'NN'), ('business', 'NN'), ('wire'...",96.87,Industrials,Aerospace & Defense,0.359367
8063,RWE,general,2022-06-15,energyvoice,"['RenewableEnergy', 'GreenHydrogen', 'Recruiti...",0,RWE,BP to boost hydrogen team as energy giant seek...,bp lon bp beef hydrogen team according news re...,"['bp', 'lon', 'bp', 'beef', 'hydrogen', 'team'...",['bp lon bp beef hydrogen team according news ...,"[('bp', 'NN'), ('lon', 'NN'), ('bp', 'NN'), ('...","[[('bp', 'NN'), ('lon', 'NN'), ('bp', 'NN'), (...",29.97,Utilities,Utilities—Diversified,0.086800
5612,Heidelberg Cement,business,2022-02-11,worldcement,"['ClimateChange', 'ValueChain', 'Governance', ...",0,HEI,CDP celebrates HeidelbergCement as ‘ Supplier ...,instruction enable javascript web browser foll...,"['instruction', 'enable', 'javascript', 'web',...","['', 'instructions enable javascript web brows...","[('instruction', 'NN'), ('enable', 'JJ'), ('ja...","[[], [('enable', 'JJ'), ('javascript', 'NN'), ...",11.91,Basic Materials,Building Materials,0.399864
3797,Deutsche Bank,NaN,2021-12-15,wsj,"['EMobility', 'Governance', 'Vaccine']",0,DBK,How Bad Can Inflation Be? Turkey Offers a Warn...,turkey rate currently offer warning soaring in...,"['turkey', 'rate', 'currently', 'offer', 'warn...","['turkey rate currently offers warning', 'soar...","[('turkey', 'JJ'), ('rate', 'NN'), ('currently...","[[('turkey', 'JJ'), ('rate', 'NN'), ('currentl...",24.97,Financials,Banks,0.129217
7053,Porsche,esg,2022-05-06,esgtoday,"['EMobility', 'LowCarbon']",0,PAH3,Porsche Leads $ 400 Million Capital Raise for ...,series funding round led luxury sport car manu...,"['series', 'funding', 'round', 'led', 'luxury'...",['series funding round led luxury sports car m...,"[('series', 'NN'), ('funding', 'NN'), ('round'...","[[('series', 'NN'), ('funding', 'NN'), ('round...",16.65,Consumer Discretionary,Auto Manufacturers,0.238658
4185,Deutsche Bank,business,2021-11-09,cnbc,"['GreatResignation', 'Recruiting', 'Environment']",0,DBK,Switching jobs can lead to higher pay. Here's ...,forget negotiating current employer sometimes ...,"['forget', 'negotiating', 'current', 'employer...","['', 'forget negotiating current employer', 's...","[('forget', 'VB'), ('negotiating', 'VBG'), ('c...","[[], [('forget', 'VB'), ('negotiating', 'VBG')...",24.97,Financials,Banks,0.328927
6380,MTU,general,2021-01-01,irishexaminer,"['HumanCapital', 'GenderDiversity']",0,MTX,A new dawn as CIT and ITT become Munster Techn...,academic council member attendee cit itt pictu...,"['academic', 'council', 'member', 'attendee', ...",['academic council members attendees cit itt p...,"[('academic', 'JJ'), ('council', 'NN'), ('memb...","[[('academic', 'JJ'), ('council', 'NN'), ('cit...",12.24,Industrials,Aerospace & Defense,0.376223
7070,Porsche,general,2021-09-15,businesswire,['CustomerService'],0,PAH3,XPEL Introduces ULTIMATE PLUS™ BLACK,gloss black film selfhealing property xpel ult...,"['gloss', 'black', 'film', 'selfhealing', 'pro...",['gloss black film selfhealing properties xpel...,"[('gloss', 'NN'), ('black', 'JJ')